<a href="https://colab.research.google.com/github/ena2509/chatbot/blob/main/chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM,Dense,GlobalMaxPooling1D,Flatten 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt        

In [3]:
%%writefile content.json
{"intents":[
            {"tag":"greeting",
            "input":["Hi there","How are you","Is anyone there?","hey","Hola","Hello","Good day","hey","Hi"],
            "responses":["Hi stranger","yes,how can I help ","Ey watsupp","Hola","Hey dude","You again"]
            },
            {"tag":"goodbye",
             "input":["bye","See you later","Goodbye","ok bye", "bye bye"],
             "responses":["See you!","Have a nice day!","Sure Bye","Later Dude","Peace out"]
            },
            {"tag":"thanks",
              "input":["Thanks","Thankyou","Thats helpful","awesome,thanks","Thanks for helping me"],
              "responses":["Happy to help!","Any time!","My pleasure","You are Welcome"]
            },
            {"tag":"noanswer",
               "input":[],
               "responses":["Sorry,can't understand you","please give me more info"]
            },
            {"tag":"name",
                "input":["What is your name","name","Tell me your name","whats your name","Who are you"],
                "responses":["I'm a bot", "bot"]
            },
            {"tag":"options",
                 "input":["What do you do","How can you help","What do you know"],
                 "responses":["Talk to me","ask me anything"]
            }
          ]

}

Overwriting content.json


In [4]:
#Importing the dataset
with open('content.json') as content:
  data1=json.load(content)

In [5]:
#getting all the data to lists
tags=[]
inputs=[]
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [6]:
#converting to dataframe
data=pd.DataFrame({"inputs":inputs,
                   "tags":tags})

In [7]:
# printing the data
data

,inputs,tags
0,Hi there,greeting
1,How are you,greeting
2,Is anyone there?,greeting
3,hey,greeting
4,Hola,greeting
5,Hello,greeting
6,Good day,greeting
7,hey,greeting
8,Hi,greeting
9,bye,goodbye


In [8]:
#preprocessing such as removing the punctuations, converting to lowercase, encoding the textual data to numeric data
import string
data['inputs']=data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs']=data['inputs'].apply(lambda wrd:''.join(wrd))

In [9]:
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=200)
tokenizer.fit_on_texts(data['inputs'])
train=tokenizer.texts_to_sequences(data['inputs'])
#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train=pad_sequences(train,maxlen=5)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(data['tags'])

In [10]:
input_shape=x_train.shape[1]
print(input_shape)

5


In [11]:
#define vocabulary
vocabulary=len(tokenizer.word_index)
print("number of unique words:",vocabulary)
output_length=le.classes_.shape[0]
print("output length:",output_length)

number of unique words: 35
output length: 5


In [12]:
#Creating the model

i=Input(shape=(input_shape,))
x=Embedding(vocabulary+1,10)(i)
x=LSTM(10,return_sequences=True)(x)
x=Flatten()(x)
x=Dense(output_length,activation='softmax')(x)
model=Model(i,x)

In [13]:
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [14]:
#training the model
train=model.fit(x_train,y_train,epochs=100)

Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 1.6108 - accuracy: 0.1111
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 1.6094 - accuracy: 0.1481
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 1.6081 - accuracy: 0.2963
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 1.6068 - accuracy: 0.3333
Epoch 5/100
1/1 [==============================] - 0s 21ms/step - loss: 1.6055 - accuracy: 0.3333
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 1.6043 - accuracy: 0.3333
Epoch 7/100
1/1 [==============================] - 0s 24ms/step - loss: 1.6030 - accuracy: 0.3333
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6017 - accuracy: 0.3333
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - loss: 1.6004 - accuracy: 0.3333
Epoch 10/100
1/1 [==============================] - 0s 20ms/step - loss: 1.5991 - accuracy: 0.3333
Epoch 11/100
1/1 [===

In [17]:
#Chat
import random

while True:
  texts_p=[]
  prediction_input=input('You:')

  #removing punctuation and converting to lowercase
  prediction_input=[letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input=''.join(prediction_input)
  texts_p.append(prediction_input)

  #tokenizing and padding
  prediction_input=tokenizer.texts_to_sequences(texts_p)
  prediction_input=np.array(prediction_input).reshape(-1)
  prediction_input=pad_sequences([prediction_input],input_shape)

  #getting output form the model
  output=model.predict(prediction_input)
  output=output
  
  #finding the right tag and predicting
  response_tag=le.inverse_transform([output])
  print('Bot:',random.choice(responses[response_tag]))
  if response_tag=='goodbye':
    break


You:hi


ValueError: ignored